In [ ]:
import pandas as pd
import re

# Load the dataset (reusing loading logic for robustness)
try:
    df = pd.read_csv('final_food_delivery_dataset.csv')
except:
    # Fallback to regeneration
    orders_df = pd.read_csv('orders.csv')
    users_df = pd.read_json('users.json')
    restaurant_data = []
    with open('restaurants.sql', 'r') as f:
        for line in f:
            if line.strip().startswith("INSERT INTO restaurants VALUES"):
                match = re.search(r"\((.*)\);", line)
                if match:
                    restaurant_data.append([p.strip().strip("'") for p in match.group(1).split(',')])
    restaurants_df = pd.DataFrame(restaurant_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
    restaurants_df['restaurant_id'] = pd.to_numeric(restaurants_df['restaurant_id'])
    merged = pd.merge(orders_df, users_df, on='user_id', how='left')
    df = pd.merge(merged, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_details'))

# Filter for Gold members
gold_df = df[df['membership'] == 'Gold']

# Find top revenue city among Gold members
city_revenue = gold_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
top_city = city_revenue.index[0]
top_revenue = city_revenue.iloc[0]

# Count orders in that city for Gold members
orders_in_top_city = gold_df[gold_df['city'] == top_city].shape[0]

print(f"Top Revenue City (Gold): {top_city}")
print(f"Revenue: {top_revenue}")
print(f"Number of orders in {top_city} by Gold members: {orders_in_top_city}")